# NYC 311 Service Requests from 2010 to present - Data Profiling and Data Cleaning Examples


The dataset that is used for all the examples is the [311 Service Requests from 2010 to Present](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9), which contains 311 calls or service requests made from 2010 to present.

The dataset consists of over 27 million rows with information about the time, complaint type, location and status of the 311 calls. The d
The dataset was accessed with the [Socrata Open Data API (SODA)](https://dev.socrata.com/).

In [1]:
#Importing required libraries

import pandas as pd

In [2]:
#!pip install pyspark

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
#spark = SparkSession.builder.appName('data').getOrCreate()

In [6]:
from openclean.data.source.socrata import Socrata


Finding available datasets related to 311 Service Requests in 'data.cityofnewyork.us' domain

In [7]:
for dataset in Socrata().catalog(domain='data.cityofnewyork.us'):
    if '311' in dataset.name.lower() and 'service requests' in dataset.name.lower():
        print(f'{dataset.identifier}\t{dataset.domain}\t{dataset.name}')

erm2-nwe9	data.cityofnewyork.us	311 Service Requests from 2010 to Present
3rfa-3xsf	data.cityofnewyork.us	311 Service Requests for 2009
sqcr-6mww	data.cityofnewyork.us	311 Service Requests for 2004
uzcy-9puk	data.cityofnewyork.us	311 Service Requests for 2008
hy4q-igkk	data.cityofnewyork.us	311 Service Requests for 2006
sxmw-f24h	data.cityofnewyork.us	311 Service Requests for 2005
aiww-p3af	data.cityofnewyork.us	311 Service Requests for 2007


In [8]:
#Importing required libraries

import gzip
import humanfriendly
import os

In [9]:
dataset = Socrata().dataset('erm2-nwe9')

In [10]:
datafile = './erm2-nwe9.tsv.gz'

In [11]:
# Download file only if it does not exist already.

if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)

In [12]:
fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))

Using '311 Service Requests from 2010 to Present' in file ./erm2-nwe9.tsv.gz of size 2.91 GB


In [13]:
# Due to the size of the full dataset file, we make use of openclean's
# stream operator to avoid having to load the dataset into main-memory.

from openclean.pipeline import stream

ds_full = stream(datafile)

In [15]:
# Count number of records in the datasets

print(f'{ds_full.count():,} rows.')

In [16]:
# Print the first ten rows of the dataset to get a first
# idea of the content

ds_full.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,51560911,08/17/2021 07:34:45 AM,08/17/2021 09:34:29 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11421,92-13 92 AVENUE,...,,,,,,,,40.68884640295723,-73.85075384808735,"(40.68884640295723, -73.85075384808735)"
1,51560923,08/17/2021 06:04:48 PM,08/17/2021 07:25:36 PM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,10468,CLAFLIN AVENUE,...,,,,,,,,40.87023400587434,-73.90060480716431,"(40.87023400587434, -73.90060480716431)"
2,50469886,05/04/2021 11:40:00 AM,07/21/2021 01:55:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,,11214,,...,,,,,,,,40.60913860285155,-74.00185479490075,"(40.60913860285155, -74.00185479490075)"
3,50470191,05/04/2021 04:31:32 PM,,DPR,Department of Parks and Recreation,New Tree Request,For One Address,Street,11229,2041 EAST 18 STREET,...,,,,,,,,40.600422830424975,-73.95378022131071,"(40.600422830424975, -73.95378022131071)"
4,50470218,05/05/2021 01:47:05 AM,10/01/2021 10:53:17 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Signs of Rodents,1-2 Family Dwelling,11420,115-44 135 STREET,...,,,,,,,,40.67997699346374,-73.8059047916443,"(40.67997699346374, -73.8059047916443)"
5,51561558,08/17/2021 10:09:21 AM,09/20/2021 12:00:00 AM,DOB,Department of Buildings,Building/Use,Illegal Conversion Of Residential Building/Space,,11420,130-47 115 STREET,...,,,,,,,,40.67453774844052,-73.82351342734557,"(40.67453774844052, -73.82351342734557)"
6,50470376,05/04/2021 08:17:49 PM,05/04/2021 08:30:42 PM,NYPD,New York City Police Department,Abandoned Vehicle,With License Plate,Street/Sidewalk,10469,1111 WARING AVENUE,...,,,,,,,,40.861134246791956,-73.85544294948706,"(40.861134246791956, -73.85544294948706)"
7,51561576,08/17/2021 07:45:48 PM,08/17/2021 08:39:41 PM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10456,1297 CLAY AVENUE,...,,,,,,,,40.8349030100631,-73.90918607978982,"(40.8349030100631, -73.90918607978982)"
8,50470422,05/04/2021 06:57:00 AM,05/14/2021 07:15:00 AM,DEP,Department of Environmental Protection,Air Quality,"Air: Odor/Fumes, Vehicle Idling (AD3)",,11217,21 ST JOHNS PLACE,...,,,,,,,,40.67809928553341,-73.9782926831807,"(40.67809928553341, -73.9782926831807)"
9,51561680,08/17/2021 05:08:00 PM,08/18/2021 09:46:34 AM,DOT,Department of Transportation,Broken Parking Meter,Coin or Card Did Not Register,Street,,,...,,,,,,,,,,


In [17]:
ds_full.head(5)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,51560911,08/17/2021 07:34:45 AM,08/17/2021 09:34:29 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11421,92-13 92 AVENUE,...,,,,,,,,40.68884640295723,-73.85075384808735,"(40.68884640295723, -73.85075384808735)"
1,51560923,08/17/2021 06:04:48 PM,08/17/2021 07:25:36 PM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,10468,CLAFLIN AVENUE,...,,,,,,,,40.87023400587434,-73.90060480716431,"(40.87023400587434, -73.90060480716431)"
2,50469886,05/04/2021 11:40:00 AM,07/21/2021 01:55:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,,11214,,...,,,,,,,,40.60913860285155,-74.00185479490075,"(40.60913860285155, -74.00185479490075)"
3,50470191,05/04/2021 04:31:32 PM,,DPR,Department of Parks and Recreation,New Tree Request,For One Address,Street,11229,2041 EAST 18 STREET,...,,,,,,,,40.600422830424975,-73.95378022131071,"(40.600422830424975, -73.95378022131071)"
4,50470218,05/05/2021 01:47:05 AM,10/01/2021 10:53:17 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Signs of Rodents,1-2 Family Dwelling,11420,115-44 135 STREET,...,,,,,,,,40.67997699346374,-73.8059047916443,"(40.67997699346374, -73.8059047916443)"


# Create a view on a subset of columns in the dataset

Subset of data is created to run tests and profile data for the project. Full dataset will be used for further analysis.

In [18]:
ds_subset = ds_full.head(5000000)

In [19]:
ds_subset = stream(ds_subset)

In [20]:
#Check the data type of created data object

type(ds_subset)

openclean.pipeline.DataPipeline

In [21]:
print(f'{ds_subset.count():,} rows.')

5,000,000 rows.


In [22]:
ds_subset.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,51560911,08/17/2021 07:34:45 AM,08/17/2021 09:34:29 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11421,92-13 92 AVENUE,...,,,,,,,,40.68884640295723,-73.85075384808735,"(40.68884640295723, -73.85075384808735)"
1,51560923,08/17/2021 06:04:48 PM,08/17/2021 07:25:36 PM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,10468,CLAFLIN AVENUE,...,,,,,,,,40.87023400587434,-73.90060480716431,"(40.87023400587434, -73.90060480716431)"
2,50469886,05/04/2021 11:40:00 AM,07/21/2021 01:55:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,,11214,,...,,,,,,,,40.60913860285155,-74.00185479490075,"(40.60913860285155, -74.00185479490075)"
3,50470191,05/04/2021 04:31:32 PM,,DPR,Department of Parks and Recreation,New Tree Request,For One Address,Street,11229,2041 EAST 18 STREET,...,,,,,,,,40.600422830424975,-73.95378022131071,"(40.600422830424975, -73.95378022131071)"
4,50470218,05/05/2021 01:47:05 AM,10/01/2021 10:53:17 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Signs of Rodents,1-2 Family Dwelling,11420,115-44 135 STREET,...,,,,,,,,40.67997699346374,-73.8059047916443,"(40.67997699346374, -73.8059047916443)"
5,51561558,08/17/2021 10:09:21 AM,09/20/2021 12:00:00 AM,DOB,Department of Buildings,Building/Use,Illegal Conversion Of Residential Building/Space,,11420,130-47 115 STREET,...,,,,,,,,40.67453774844052,-73.82351342734557,"(40.67453774844052, -73.82351342734557)"
6,50470376,05/04/2021 08:17:49 PM,05/04/2021 08:30:42 PM,NYPD,New York City Police Department,Abandoned Vehicle,With License Plate,Street/Sidewalk,10469,1111 WARING AVENUE,...,,,,,,,,40.861134246791956,-73.85544294948706,"(40.861134246791956, -73.85544294948706)"
7,51561576,08/17/2021 07:45:48 PM,08/17/2021 08:39:41 PM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10456,1297 CLAY AVENUE,...,,,,,,,,40.8349030100631,-73.90918607978982,"(40.8349030100631, -73.90918607978982)"
8,50470422,05/04/2021 06:57:00 AM,05/14/2021 07:15:00 AM,DEP,Department of Environmental Protection,Air Quality,"Air: Odor/Fumes, Vehicle Idling (AD3)",,11217,21 ST JOHNS PLACE,...,,,,,,,,40.67809928553341,-73.9782926831807,"(40.67809928553341, -73.9782926831807)"
9,51561680,08/17/2021 05:08:00 PM,08/18/2021 09:46:34 AM,DOT,Department of Transportation,Broken Parking Meter,Coin or Card Did Not Register,Street,,,...,,,,,,,,,,


# Data Profiling

In [23]:
from openclean.profiling.column import DefaultColumnProfiler

In [26]:
#profiles = ds_full.profile(default_profiler=DefaultColumnProfiler)

#profiles.stats()

In [27]:
# Profile the resulting dataset view using the default data profiler
profiles1 = ds_subset.profile(default_profiler=DefaultColumnProfiler)

# Print overview of profiling results
profiles1.stats()

/opt/homebrew/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PLACE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/opt/homebrew/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname VE identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,total,empty,distinct,uniqueness,entropy
Unique Key,5000000,0,5000000,1.000000,22.253497
Created Date,5000000,0,4421707,0.884341,21.786082
Closed Date,5000000,201273,3289215,0.685435,20.751632
Agency,5000000,0,28,0.000006,3.133487
Agency Name,5000000,0,1554,0.000311,3.398212
Complaint Type,5000000,0,312,0.000062,5.784354
Descriptor,5000000,5026,1558,0.000312,7.313881
Location Type,5000000,1429454,171,0.000048,3.712247
Incident Zip,5000000,742492,2082,0.000489,7.302872
Incident Address,5000000,1395596,812911,0.225533,18.192463


We see some columns with extremely large amounts of null values. The column values seem to be specify for certain incidents and requests. 
We also find numerous columns with repetitive or non-distinct data, such columns could be termed as class related columns. These columns have values which belong to different classes.
We will try to reduce the null values in the data and avoid duplication.



In [29]:
# Print the most frequent data type for each column.
print('Schema\n------')
for col in ds_full.columns:
    p = profiles1.column(col)
    print("  '{}' ({})".format(col, p['datatypes']['distinct'].most_common(1)[0][0]))

Schema
------
  'Unique Key' (int)
  'Created Date' (date)
  'Closed Date' (date)
  'Agency' (str)
  'Agency Name' (str)
  'Complaint Type' (str)
  'Descriptor' (str)
  'Location Type' (str)
  'Incident Zip' (int)
  'Incident Address' (str)
  'Street Name' (str)
  'Cross Street 1' (str)
  'Cross Street 2' (str)
  'Intersection Street 1' (str)
  'Intersection Street 2' (str)
  'Address Type' (str)
  'City' (str)
  'Landmark' (str)
  'Facility Type' (str)
  'Status' (str)
  'Due Date' (date)
  'Resolution Description' (str)
  'Resolution Action Updated Date' (date)
  'Community Board' (str)
  'BBL' (int)
  'Borough' (str)
  'X Coordinate (State Plane)' (int)
  'Y Coordinate (State Plane)' (int)
  'Open Data Channel Type' (str)
  'Park Facility Name' (str)
  'Park Borough' (str)
  'Vehicle Type' (str)
  'Taxi Company Borough' (str)
  'Taxi Pick Up Location' (str)
  'Bridge Highway Name' (str)
  'Bridge Highway Direction' (str)
  'Road Ramp' (str)
  'Bridge Highway Segment' (str)
  'Latitu

In [30]:
# Print the most frequent data type for column Created Date.

profiles1.minmax('Created Date')

,min,max
date,2010-01-01,2021-09-23 00:56:44


In [31]:
# Print the most frequent data type for column Closed Date.

profiles1.minmax('Closed Date')

,min,max
date,1899-12-31 19:00:00,2201-06-17


In [32]:
# Print the most frequent data type for column Due Date.

profiles1.minmax('Due Date')

,min,max
date,1900-01-02,2021-10-22 09:50:05


In [33]:
# Print the most frequent data type for column BBL.

profiles1.minmax('BBL')

,min,max
int,0,6066007501


In [34]:
# Print the most frequent data type for column Taxi Pick Up Location.

profiles1.minmax('Taxi Pick Up Location')

,min,max
str,- 6104 HIGHLAND PLACE NOT WITHIN NEW YORK CITY,YORK AVENUE BETWEEN EAST 74 STREET AND EAST ...


In [35]:
# Print the most frequent data type for column Location.

profiles1.minmax('Location')

,min,max
str,"(40.49862773316935, -74.24045188891107)","(40.91294056699566, -73.90628271248819)"


In [36]:
# Print the most frequent data type for column Agency.

profiles1.minmax('Agency')

,min,max
str,3-1-1,TLC


We remove Columns with high amount of null values.

In [42]:
COLUMNS = [
    'Unique Key',
  'Created Date',
  'Closed Date', 
  'Agency', 
  'Agency Name', 
  'Complaint Type',
  'Descriptor', 
  'Location Type',
  'Incident Zip', 
  'Incident Address', 
  'Street Name', 
  'Cross Street 1', 
  'Cross Street 2', 
  'Address Type', 
  'City', 
  'Facility Type', 
  'Status', 
  'Resolution Description', 
  'Resolution Action Updated Date', 
  'Community Board', 
  'BBL',
  'Borough', 
  'X Coordinate (State Plane)', 
  'Y Coordinate (State Plane)', 
  'Open Data Channel Type', 
  'Park Facility Name',
  'Park Borough',
  'Latitude',
  'Longitude',
  'Location' 
]

ds = ds_subset.select(columns=COLUMNS)


As seen in the output below, the dataset head has comparatively less null values and does not have any redundant columns

In [43]:
ds.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Latitude,Longitude,Location
0,51560911,08/17/2021 07:34:45 AM,08/17/2021 09:34:29 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11421,92-13 92 AVENUE,...,4089860017,QUEENS,1025640,190277,MOBILE,Unspecified,QUEENS,40.68884640295723,-73.85075384808735,"(40.68884640295723, -73.85075384808735)"
1,51560923,08/17/2021 06:04:48 PM,08/17/2021 07:25:36 PM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,10468,CLAFLIN AVENUE,...,,BRONX,1011740,256343,PHONE,Unspecified,BRONX,40.87023400587434,-73.90060480716431,"(40.87023400587434, -73.90060480716431)"
2,50469886,05/04/2021 11:40:00 AM,07/21/2021 01:55:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,,11214,,...,,BROOKLYN,983735,161202,UNKNOWN,Unspecified,BROOKLYN,40.60913860285155,-74.00185479490075,"(40.60913860285155, -74.00185479490075)"
3,50470191,05/04/2021 04:31:32 PM,,DPR,Department of Parks and Recreation,New Tree Request,For One Address,Street,11229,2041 EAST 18 STREET,...,3073230065,BROOKLYN,997085,158030,UNKNOWN,Unspecified,BROOKLYN,40.600422830424975,-73.95378022131071,"(40.600422830424975, -73.95378022131071)"
4,50470218,05/05/2021 01:47:05 AM,10/01/2021 10:53:17 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Signs of Rodents,1-2 Family Dwelling,11420,115-44 135 STREET,...,4116770025,QUEENS,1038085,187070,MOBILE,Unspecified,QUEENS,40.67997699346374,-73.8059047916443,"(40.67997699346374, -73.8059047916443)"
5,51561558,08/17/2021 10:09:21 AM,09/20/2021 12:00:00 AM,DOB,Department of Buildings,Building/Use,Illegal Conversion Of Residential Building/Space,,11420,130-47 115 STREET,...,4117220013,QUEENS,1033205,185078,UNKNOWN,Unspecified,QUEENS,40.67453774844052,-73.82351342734557,"(40.67453774844052, -73.82351342734557)"
6,50470376,05/04/2021 08:17:49 PM,05/04/2021 08:30:42 PM,NYPD,New York City Police Department,Abandoned Vehicle,With License Plate,Street/Sidewalk,10469,1111 WARING AVENUE,...,2044530003,BRONX,1024236,253045,PHONE,Unspecified,BRONX,40.861134246791956,-73.85544294948706,"(40.861134246791956, -73.85544294948706)"
7,51561576,08/17/2021 07:45:48 PM,08/17/2021 08:39:41 PM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10456,1297 CLAY AVENUE,...,2027820146,BRONX,1009380,243468,ONLINE,Unspecified,BRONX,40.8349030100631,-73.90918607978982,"(40.8349030100631, -73.90918607978982)"
8,50470422,05/04/2021 06:57:00 AM,05/14/2021 07:15:00 AM,DEP,Department of Environmental Protection,Air Quality,"Air: Odor/Fumes, Vehicle Idling (AD3)",,11217,21 ST JOHNS PLACE,...,3009440075,BROOKLYN,990271,186327,ONLINE,Unspecified,BROOKLYN,40.67809928553341,-73.9782926831807,"(40.67809928553341, -73.9782926831807)"
9,51561680,08/17/2021 05:08:00 PM,08/18/2021 09:46:34 AM,DOT,Department of Transportation,Broken Parking Meter,Coin or Card Did Not Register,Street,,,...,,Unspecified,,,ONLINE,Unspecified,Unspecified,,,


In [44]:
type(ds)

openclean.pipeline.DataPipeline

In [45]:
ds.columns

['Unique Key',
 'Created Date',
 'Closed Date',
 'Agency',
 'Agency Name',
 'Complaint Type',
 'Descriptor',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Cross Street 1',
 'Cross Street 2',
 'Address Type',
 'City',
 'Facility Type',
 'Status',
 'Resolution Description',
 'Resolution Action Updated Date',
 'Community Board',
 'BBL',
 'Borough',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)',
 'Open Data Channel Type',
 'Park Facility Name',
 'Park Borough',
 'Latitude',
 'Longitude',
 'Location']

# Outliers and Anomalies in Dataset Columns.

We try to look for outliers in data, data values which may have very high or extremely low frequency. Such values may affect future predictions and alter overall behaviour of data.

Get set of distinct values for columns 'Borough','Park Borough','Agency','Address Type','Facility Type','Status','Community Board' . Print the values in decreasing order of frequency.

In [46]:
boroughs = ds.distinct('Borough')
for rank, val in enumerate(boroughs.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   BROOKLYN   1,381,026
2   QUEENS   1,278,847
3   MANHATTAN   1,114,528
4   BRONX     759,217
5   STATEN ISLAND     262,599
6   Unspecified     184,798
7         18,985


In [47]:
park_boroughs = ds.distinct('Park Borough')
for rank, val in enumerate(park_boroughs.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   BROOKLYN   1,381,026
2   QUEENS   1,278,847
3   MANHATTAN   1,114,528
4   BRONX     759,217
5   STATEN ISLAND     262,599
6   Unspecified     184,798
7         18,985


As seen in the above results for Boroughs and Park Boroughs, both columns represent identical data. We disregard one of the columns from our dataset to avoid duplicate columns.

In [48]:
agency = ds.distinct('Agency')
for rank, val in enumerate(agency.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   NYPD   1,582,666
2   DOT     856,536
3   DEP     564,128
4   DPR     478,914
5   DOB     300,224
6   DOF     246,555
7   DOHMH     230,721
8   HPD     220,844
9   DSNY     151,479
10  TLC     112,744
11  DCA      90,899
12  DFTA      46,644
13  HRA      44,441
14  MAYORâS OFFICE OF SPECIAL ENFORCEMENT      21,407
15  3-1-1      19,533
16  DOE       9,559
17  EDC       7,208
18  DHS       6,657
19  NYCEM       4,479
20  DOITT       4,197
21  DCAS          53
22  FDNY          44
23  TAX          28
24  ACS          25
25  DORIS           9
26  DVS           3
27  DCP           2
28  TAT           1


In [49]:
addtype = ds.distinct('Address Type')
for rank, val in enumerate(addtype.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   ADDRESS   3,035,397
2   INTERSECTION     903,909
3        857,324
4   BLOCKFACE     172,877
5   LATLONG      24,232
6   PLACENAME       6,171
7   UNRECOGNIZED          90


In [50]:
factype = ds.distinct('Facility Type')
for rank, val in enumerate(factype.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   N/A   2,956,306
2   Precinct   1,306,464
3        698,176
4   DSNY Garage      30,405
5   School       7,012
6   School District       1,637


In [51]:
status = ds.distinct('Status')
for rank, val in enumerate(status.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   Closed   4,717,828
2   Pending     101,042
3   Assigned      70,134
4   Open      60,180
5   In Progress      50,066
6   Started         596
7   Email Sent         112
8   Unassigned          21
9   Closed - Testing          15
10  Draft           4
11  Unspecified           2


In [52]:
commboard = ds.distinct('Community Board')
for rank, val in enumerate(commboard.most_common()):
    st, freq = val
    print(f'{rank + 1:<3} {st}  {freq:>10,}')

1   0 Unspecified     184,798
2   Unspecified MANHATTAN     160,938
3   Unspecified BROOKLYN     142,992
4   Unspecified QUEENS     136,112
5   12 QUEENS     118,864
6   07 QUEENS     118,272
7   12 MANHATTAN     116,691
8   Unspecified BRONX     106,528
9   01 BROOKLYN     105,490
10  01 QUEENS     100,728
11  05 QUEENS     100,643
12  03 MANHATTAN      98,596
13  12 BRONX      95,826
14  13 QUEENS      95,768
15  07 MANHATTAN      95,506
16  01 STATEN ISLAND      93,683
17  05 MANHATTAN      91,614
18  18 BROOKLYN      91,551
19  09 QUEENS      91,423
20  08 MANHATTAN      90,073
21  03 BROOKLYN      83,250
22  03 STATEN ISLAND      82,836
23  15 BROOKLYN      82,222
24  05 BROOKLYN      82,064
25  02 MANHATTAN      81,924
26  14 BROOKLYN      80,228
27  04 MANHATTAN      78,938
28  12 BROOKLYN      78,723
29  10 QUEENS      76,770
30  08 QUEENS      76,726
31  11 BROOKLYN      72,611
32  10 MANHATTAN      71,137
33  03 QUEENS      71,104
34  17 BROOKLYN      70,878
35  09 BRONX     

As seen in the above results, each columns has certain values which may be considered as outliers. These outlier could be used in further analysis for detecting specific or special instances and situations.

In [53]:
#Removing Park Borough from the list of columns.

COLUMNS = [
    'Unique Key',
  'Created Date',
  'Closed Date', 
  'Agency', 
  'Agency Name', 
  'Complaint Type',
  'Descriptor', 
  'Location Type',
  'Incident Zip', 
  'Incident Address', 
  'Street Name', 
  'Cross Street 1', 
  'Cross Street 2', 
  'Address Type', 
  'City', 
  'Facility Type', 
  'Status', 
  'Resolution Description', 
  'Resolution Action Updated Date', 
  'Community Board', 
  'BBL',
  'Borough', 
  'X Coordinate (State Plane)', 
  'Y Coordinate (State Plane)', 
  'Open Data Channel Type', 
  'Park Facility Name',
  'Latitude',
  'Longitude',
  'Location' 
]

ds = ds_subset.select(columns=COLUMNS)



In [54]:
ds.columns

['Unique Key',
 'Created Date',
 'Closed Date',
 'Agency',
 'Agency Name',
 'Complaint Type',
 'Descriptor',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Cross Street 1',
 'Cross Street 2',
 'Address Type',
 'City',
 'Facility Type',
 'Status',
 'Resolution Description',
 'Resolution Action Updated Date',
 'Community Board',
 'BBL',
 'Borough',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)',
 'Open Data Channel Type',
 'Park Facility Name',
 'Latitude',
 'Longitude',
 'Location']

# Data Cleaning

Reformatting values for consistency in columns and removing Null values

In [55]:
dstemp = ds.head(5000000)

In [56]:
type(dstemp)

pandas.core.frame.DataFrame

In [57]:
dstemp.Agency.unique()

array(['NYPD', 'DOT', 'DPR', 'DOHMH', 'DOB', 'DEP', 'DSNY', 'DOF', 'DHS',
       'HPD', 'EDC', 'DCA', 'TLC', 'DOITT', 'DFTA', 'DOE', 'TAX', 'DCAS',
       'TAT', 'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT', 'ACS',
       'DVS', 'NYCEM', 'HRA', 'FDNY', 'DCP', 'DORIS', '3-1-1'],
      dtype=object)

In [58]:
def update_MOSE(Agency):

  """Takes in an agency name and converts to abbreviated form, if the agency is
     the Office of Special Enforcement."""

  if Agency == 'MAYORâ\x80\x99S OFFICE OF SPECIAL ENFORCEMENT':
      return "MOSE"
  else:
      return Agency
    
dstemp.Agency = dstemp.Agency.map(lambda Agency: update_MOSE(Agency))

In [59]:
dstemp.Agency.unique()

array(['NYPD', 'DOT', 'DPR', 'DOHMH', 'DOB', 'DEP', 'DSNY', 'DOF', 'DHS',
       'HPD', 'EDC', 'DCA', 'TLC', 'DOITT', 'DFTA', 'DOE', 'TAX', 'DCAS',
       'TAT', 'MOSE', 'ACS', 'DVS', 'NYCEM', 'HRA', 'FDNY', 'DCP',
       'DORIS', '3-1-1'], dtype=object)

In [ ]:
#dstemp['Facility Type'].unique()

In [ ]:
#dstemp.head()

#Create a function to replace N/A values with Unspecified to avoid confusion between null and N/A values for Facility Type column

def update_NA(dstemp['Facility Type']):

  if dstemp['Facility Type'] == 'N/A':
      return "Unspecified"
  else:
      return dstemp['Facility Type']
    
dstemp['Facility Type'] = dstemp['Facility Type'].map(lambda dstemp['Facility Type']: update_NA(dstemp['Facility Type']))

# Remove NA Values

Remove rows from the dataset if Null values exceed a specified threshold for the given row.
For our example, we consider..
Threshold = 50%

If Null values in a row exceed more than 50% of the columns, the row will be discarded

In [60]:
threshold = 50.0
min_count =  int(((100-threshold)/100)*dstemp.shape[1] + 1)
dstemp1 = dstemp.dropna( axis=0, 
                    thresh=min_count)

In [61]:
dstemp1.shape

(5000000, 29)

The above output shows there were no rows found with Null values > 50% in a row.

In [68]:
ds = stream(dstemp1)

# Outlier Detection using scikit-learn

Run through columns to check for values which may have a comparatively higher or extremely lower frequency as compared to other values in the column.
This step gives us an idea about the behaviour and distribution of values as well as helps us in detecting outlier which could be skipped in certain predictions to avoid bias.

In [66]:
from openclean.profiling.anomalies.sklearn import DBSCANOutliers

In [69]:
complaint_type = ds.distinct('Complaint Type')
incident_zip = ds.distinct('Incident Zip')
city = ds.distinct('City')
res_decp = ds.distinct('Resolution Description')

In [70]:
# Print the ten most frequent values for the 'Complaint Type' column

for rank, val in enumerate(complaint_type.most_common(10)):
    dt, freq = val
    print(f'{rank + 1:<3} {dt:>8}  {freq:>10,}')

print('\nTotal number of distinct values is {}'.format(len(complaint_type)))

1   Noise - Residential     627,924
2   Blocked Driveway     264,057
3   Water System     219,986
4   Street Light Condition     215,237
5   Street Condition     204,624
6   Illegal Parking     200,196
7   General Construction/Plumbing     179,483
8   Damaged Tree     173,800
9      Sewer     134,152
10     Noise     127,515

Total number of distinct values is 312


In [71]:
DBSCANOutliers(eps=0.01).find(complaint_type)

['Broken Parking Meter',
 'Fire Alarm - New System',
 'ELECTRIC',
 'DOF Parking - Tax Exemption',
 'Illegal Parking',
 'Special Natural Area District (SNAD)',
 'Water Conservation',
 'Green Taxi Report',
 'Question',
 'For Hire Vehicle Complaint',
 'Home Repair',
 'Discipline and Suspension',
 'Miscellaneous Categories',
 'Housing - Low Income Senior',
 'Lifeguard',
 'Illegal Animal Kept as Pet',
 'Street Light Condition',
 'ATF',
 'Unsanitary Condition',
 'Facades',
 'Illegal Fireworks',
 'Violation of Park Rules',
 'FLOORING/STAIRS',
 'PLUMBING',
 'Mass Gathering Complaint',
 'Case Management Agency Complaint',
 'Unlicensed Dog',
 'OEM Disabled Vehicle',
 'Mobile Food Vendor',
 'City Vehicle Placard Complaint',
 'Root/Sewer/Sidewalk Condition',
 'Homeless Encampment',
 'Single Occupancy Bathroom',
 'CONSTRUCTION',
 'DOF Parking - Payment Issue',
 'Sweeping/Inadequate',
 'Collection Truck Noise',
 'Highway Sign - Missing',
 'PAINT/PLASTER',
 'DCA / DOH New License Application Request'

In [72]:
# Print the ten most frequent values for the 'Incident Zip' column

for rank, val in enumerate(incident_zip.most_common(10)):
    dt, freq = val
    print(f'{rank + 1:<3} {dt:>8}  {freq:>10,}')

print('\nTotal number of distinct values is {}'.format(len(incident_zip)))

1                742,492
2      11385      55,373
3      11226      53,099
4      10466      51,233
5      10467      49,859
6      11234      48,787
7      11207      45,978
8      11208      43,944
9      11211      43,832
10     10314      42,567

Total number of distinct values is 2083


In [73]:
DBSCANOutliers(eps=0.01).find(incident_zip)

['',
 'NJ 07114',
 'UNK',
 '*',
 '1*9138',
 'IDK',
 'V6J4X9',
 'MEXICO7500',
 '11',
 '11797 9004',
 'UNKOWN',
 '98036-1542',
 '55555',
 '999999',
 '031546',
 "DON'T KNOW",
 '4009-0517',
 '1735-9100',
 'L7R1H2',
 'NJ 07310',
 '14450 0457',
 'N/A',
 '.',
 '46',
 '000000000',
 'NOT SURE',
 '11226',
 'UNKNOWN',
 '115I0',
 'NTY',
 '920469046',
 'NA',
 '10467',
 '10459',
 '171052461',
 '0000 0000',
 '11111',
 'ANONYMOUS',
 'NO CLUE',
 'MN55439',
 '11211',
 'NEWARK AIR',
 'N5X3A6',
 '?',
 '01810-1008',
 '00000',
 'LE4',
 '171111',
 '117',
 '00000000',
 'NO IDEA',
 '753570909',
 '000000',
 'NEW YORK A',
 'JFK AIRPOR',
 '47117 5524',
 '11590 5027',
 '66063-3330',
 '012345',
 'JFK',
 'NJ07114',
 '1046-0232',
 '10465',
 'X',
 'NY',
 'NOT KNOWN',
 '48393-102',
 '11414',
 '1',
 '11221',
 'AIRPORT',
 '633-0004',
 '100000',
 'NJ 07',
 '77272-2929',
 'NONE',
 '11411',
 'INDIAN WEL',
 'W1',
 '0',
 '11739100',
 'NJ',
 '00',
 'XXX',
 'XXXXX',
 '101',
 'NEWARK',
 'N/',
 '????',
 '99999',
 '328965',
 '000'

In [74]:
# Print the ten most frequent values for the 'City' column

for rank, val in enumerate(city.most_common(10)):
    dt, freq = val
    print(f'{rank + 1:<3} {dt:>8}  {freq:>10,}')

print('\nTotal number of distinct values is {}'.format(len(city)))

1   BROOKLYN   1,235,128
2   NEW YORK     954,961
3                770,621
4      BRONX     650,474
5   STATEN ISLAND     244,951
6    JAMAICA      83,881
7   FLUSHING      74,811
8    ASTORIA      62,242
9   RIDGEWOOD      43,012
10   Jamaica      32,223

Total number of distinct values is 1886


In [75]:
DBSCANOutliers(eps=0.01).find(city)

['',
 'GREAT BARRINGTON',
 'EAST ATLANTIC BEACH',
 'CO-OP CITY SECTION 5',
 'LAS  VEGAS',
 'Long Island City',
 'DANA POINT CALIFORNIA',
 'GLENN ALLEN',
 'GLEN ALLEN',
 'CORONA',
 'EAST  RUTHERFORD NJ',
 'FORT JEFFERSON STATI',
 'RONTER CENTER',
 '*',
 'SPRINGDALE',
 'FORT JEFFERSON STATION',
 'SPRINGFIELD GARDENS',
 'ATLANTA',
 'WESTHEMPSTEAD',
 'MIRAMAR',
 'LONGBEACH RD',
 'TEME',
 'GARDEN CITY PARK',
 'RIDGEWOOD',
 'NORTH MASSAPEQUA',
 'EAST SETUAKET',
 'JHACKSONVILLE',
 'COURT JEFF STA',
 'XX',
 'ST. PAUL',
 'OLD WESTBURY',
 'WILLIAM VILLE',
 'CITY OF INTERESION',
 'BLOOMINGFIELD',
 'POBOX 64886',
 'CLIFFHILL PARK,',
 'FLUSHING MEADOWS',
 'PORT JEFFERSON STA',
 'FARMERSBRANCHE',
 'N BERGEN',
 'NEW HYDE  PARK',
 'TUIPEA BUFFALO GROVE',
 'E',
 'ST. JOSEPHS VILLAGE',
 'New Hyde Park',
 'JACKSONVILLE FLORIDA',
 'MANHANTTAN',
 'HAMILTON TOWNSHIP',
 'FLUSHING',
 'MIAMI',
 'JERICHO, LI',
 'MIDDLE VILLAGE, QUEENS',
 'N.Y',
 'HUNTINGTON STATION',
 'WOODSIDE',
 'N/A',
 'JACKSON HEIGHTS',
 'C

# Data Standardization

We try creating clusters of values which may have the same meaning and context but have different representations.
For our data we use Key Collision Clustering on Street Name, Incident Address and Agency Name. Since address inputs may have inconsistencies due to manual input or varied representation of data received from different sources, our focus will be on address related attributes. 

In [76]:
# Cluster street names using 'Key Collision' clustering with the
# default fingerprint key generator.

from openclean.cluster.key import KeyCollision
from openclean.function.value.key.fingerprint import Fingerprint

In [77]:
street_names = ds.update('Street Name', str.upper).distinct('Street Name')
clusters = KeyCollision(func=Fingerprint()).clusters(street_names)

In [78]:
# Define simple helper method to print the k largest clusters.

def print_k_clusters(clusters, k=5):
    clusters = sorted(clusters, key=lambda x: len(x), reverse=True)
    val_count = sum([len(c) for c in clusters])
    print('Total number of clusters is {} with {} values'.format(len(clusters), val_count))
    for i in range(min(k, len(clusters))):
        print('\nCluster {}'.format(i + 1))
        for key, cnt in clusters[i].items():
            if key == '':
                key = "''"
            print(f'  {key} (x {cnt})')

In [79]:
print_k_clusters(clusters)

Total number of clusters is 1629 with 3300 values

Cluster 1
  '' (x 1395710)
  * (x 7)
  . (x 4)
  / (x 1)

Cluster 2
  ST JOHNS AVENUE (x 143)
  ST JOHN'S AVENUE (x 10)
  AVENUE ST JOHNS (x 38)
  AVENUE ST JOHN'S (x 1)

Cluster 3
  CENTRAL PARK W (x 19)
  W CENTRAL PARK (x 4)
  W CENTRAL PARK PARK (x 4)
  CENTRAL PARK PARK W (x 1)

Cluster 4
  CENTRAL PARK WEST PARK W (x 1)
  CENTRAL WEST PARK W (x 4)
  CENTRAL PARK WEST. PARK W (x 1)
  W CENTRAL PARK WEST PARK W (x 5)

Cluster 5
  N/A (x 516)
  NA (x 139)
  N/A/ (x 1)
  NA/ (x 1)


In [80]:
inc_add = ds.update('Incident Address', str.upper).distinct('Incident Address')
clusters2 = KeyCollision(func=Fingerprint()).clusters(inc_add)

In [81]:
agency_names = ds.update('Agency Name', str.upper).distinct('Agency Name')
clusters3 = KeyCollision(func=Fingerprint()).clusters(agency_names)

In [82]:
print_k_clusters(clusters2)

Total number of clusters is 43204 with 90546 values

Cluster 1
  N/A N/A (x 486)
  NA NA (x 133)
  N/A (x 17)
  N/A NA (x 4)
  NA (x 3)
  N/A NA/ (x 1)
  NA N/A (x 1)

Cluster 2
  12 WEST 12 STREET (x 9)
  WEST   12 STREET (x 38)
  12 WEST STREET (x 8)
  WEST 12 STREET (x 74)
  12 WEST   12 STREET (x 9)
  12 WEST WEST 12 STREET (x 1)

Cluster 3
  140 WEST STREET (x 30)
  140 WEST  140 STREET (x 10)
  WEST 140 STREET (x 57)
  WEST  140 STREET (x 7)
  140 WEST 140 STREET (x 30)
  140 WEST WEST 140 STREET (x 2)

Cluster 4
  147 WEST  230 STREET (x 13)
  230 WEST  147 STREET (x 31)
  230 WEST 147 STREET (x 47)
  147 WEST 230 STREET (x 18)
  230 WEST WEST 147 STREET (x 1)
  230 WEST WEST  147 STREET (x 1)

Cluster 5
  WEST   16 STREET (x 15)
  WEST 16 STREET (x 76)
  16 WEST   16 STREET (x 9)
  16 WEST 16 STREET (x 18)
  16 WEST STREET (x 1)
  16 WEST WEST   16 STREET (x 1)


In [83]:
print_k_clusters(clusters3)

Total number of clusters is 2 with 4 values

Cluster 1
  SCHOOL - ARCHIMEDES ACADEMY FOR MATH, SCIENCE AND TECHNOLOGY APPLICATIONS (x 1)
  SCHOOL - ARCHIMEDES ACADEMY FOR MATH, SCIENCE, AND TECHNOLOGY APPLICATIONS (x 3)

Cluster 2
  SCHOOL - SCHOLARS ACADEMY (x 2)
  SCHOOL - SCHOLARS' ACADEMY (x 1)
